i dont have any data so need to make my own!

In [11]:
# random interactions
import numpy as np, pandas as pd

n_users   = 500
n_videos  = 1_000
n_events  = 6_000         

rng = np.random.default_rng(42)

interactions = pd.DataFrame({
    "user_id" : rng.integers(0, n_users,  n_events),
    "video_id": rng.integers(0, n_videos, n_events),
    "likes"   : rng.integers(1, 10,       n_events)   # weight 1–9
})

print(interactions[:5])

   user_id  video_id  likes
0       44       619      5
1      386       829      5
2      327       640      1
3      219       594      2
4      216       467      6


In [ ]:
# 3) load the actual 1k video vectors
video_vecs = np.load("output/videos.npy")        # shape (1000, 5128)

print(video_vecs.shape)        # should be (1000, 5128)


(1000, 2348)


In [ ]:
# 4) build user vectors – code unchanged except for variable sizes
user_vecs = (
    interactions
      .assign(weight=interactions["likes"])
      .merge(pd.DataFrame({"video_id": np.arange(n_videos),
                           "vec": list(video_vecs)}), on="video_id")
      .groupby("user_id")
      .apply(lambda g: np.average(np.vstack(g["vec"]),
                                  weights=g["weight"], axis=0))
)

# 5) pad any missing users (some may have zero interactions)
user_matrix = np.zeros((n_users, 2348), dtype=np.float32)
for uid, vec in user_vecs.items():
    user_matrix[uid] = vec

# 6) save – shape is now (n_users, 5128), e.g. (500, 5128)
np.save("output/users.npy", user_matrix)


[[ 0.          0.          0.         ... -0.02604912 -0.01908159
   0.02118554]
 [ 0.          0.          0.         ... -0.02342878 -0.01793193
   0.02188703]
 [ 0.          0.          0.         ... -0.02643197 -0.0214548
   0.02315622]
 [ 0.07450291  0.          0.         ... -0.02637375 -0.02079907
   0.0213614 ]
 [ 0.          0.          0.         ... -0.03029886 -0.02247988
   0.02567782]]


C:\Users\minec\AppData\Local\Temp\ipykernel_18000\941662854.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.average(np.vstack(g["vec"]),
